In [55]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
import requests
from bs4 import BeautifulSoup
import re
import json


In [70]:

# Cấu hình trình duyệt không hiển thị (headless)
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # chạy ngầm không mở cửa sổ trình duyệt

# Tạo driver
driver = webdriver.Chrome(options=options)

# Truy cập trang
driver.get("https://suckhoedoisong.vn/tra-cuu-benh.htm")
time.sleep(3)  # đợi JS load xong

# Lấy HTML đã render đầy đủ
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Tìm danh sách bệnh
container = soup.find("div", class_="list")
diseases = []

if container:
    links = container.find_all("a", class_="item")
    for link in links:
        name = link.get_text(strip=True)
        href = link.get("href")
        if not href.startswith("http"):
            href = "https://suckhoedoisong.vn" + href
        diseases.append({
            "name": name,
            "url": href
        })

    # In 10 kết quả đầu tiên
    for d in diseases[:10]:
        print(d)

else:
    print("Không tìm thấy div.list – JS chưa load xong hoặc cấu trúc HTML đã đổi")

# Đóng trình duyệt
driver.quit()


{'name': 'Áp xe não do amip', 'url': 'https://suckhoedoisong.vn/chu-de/ap-xe-nao-do-amip-4433.htm'}
{'name': 'Ấu trùng sán lợn', 'url': 'https://suckhoedoisong.vn/chu-de/au-trung-san-lon-4422.htm'}
{'name': 'Alkapton niệu', 'url': 'https://suckhoedoisong.vn/chu-de/alkapton-nieu-4374.htm'}
{'name': 'Ấu dâm', 'url': 'https://suckhoedoisong.vn/chu-de/au-dam-4373.htm'}
{'name': 'Áp xe thận', 'url': 'https://suckhoedoisong.vn/chu-de/ap-xe-than-4372.htm'}
{'name': 'Áp xe gan do amip', 'url': 'https://suckhoedoisong.vn/chu-de/ap-xe-gan-do-amip-4367.htm'}
{'name': 'Áp xe phổi', 'url': 'https://suckhoedoisong.vn/chu-de/ap-xe-phoi-4365.htm'}
{'name': 'Áp xe hậu môn', 'url': 'https://suckhoedoisong.vn/chu-de/ap-xe-hau-mon-4356.htm'}
{'name': 'Áp xe vú', 'url': 'https://suckhoedoisong.vn/chu-de/ap-xe-vu-4292.htm'}
{'name': 'Acid uric', 'url': 'https://suckhoedoisong.vn/chu-de/acid-uric-4243.htm'}


In [12]:
def extract_all_box_category_contents(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Lỗi truy cập: {url}")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    boxes = soup.find_all("div", class_="box-category-content")

    all_articles = []
    for box in boxes:
        # Tiêu đề và link
        a_tag = box.select_one("h3.box-category-title-text > a")
        if not a_tag:
            continue
        title = a_tag.get("title")
        link = a_tag.get("href")
        if not link.startswith("http"):
            link = "https://suckhoedoisong.vn" + link

        # Mô tả ngắn
        summary_tag = box.find("p", class_="box-category-sapo")
        summary = summary_tag.get_text(strip=True) if summary_tag else ""

        all_articles.append({
            "title": title,
            "url": link,
            "summary": summary
        })

    return all_articles
url = "https://suckhoedoisong.vn/chu-de/ap-xe-nao-do-amip-4433.htm"
articles = extract_all_box_category_contents(url)

for a in articles:
    print(a)


{'title': 'Chế độ ăn cho người bị áp xe não do amip', 'url': 'https://suckhoedoisong.vn/che-do-an-cho-nguoi-bi-ap-xe-nao-do-amip-169250228174119445.htm', 'summary': 'SKĐS - Mặc dù không có khuyến nghị chế độ ăn uống cụ thể cho áp xe não do amip nhưng các tổ chức y tế uy tín đều khẳng định tầm quan trọng của dinh dưỡng hợp lý trong việc hỗ trợ hệ thống miễn dịch và sức khỏe tổng thể. Điều này rất quan trọng đối với những người bị nhiễm trùng như áp xe não do amip.'}
{'title': 'Câu hỏi thường gặp về áp xe não do amip', 'url': 'https://suckhoedoisong.vn/cau-hoi-thuong-gap-ve-ap-xe-nao-do-amip-169240926102237051.htm', 'summary': 'SKĐS - Áp xe não do amip là bệnh lý nguy hiểm do vậy cần đề cao việc phòng ngừa. Chủ yếu mọi người cần lưu ý vấn đề vệ sinh và ăn uống hàng ngày.'}
{'title': 'Áp xe não do amip: Nguyên nhân, triệu chứng, cách điều trị và phòng ngừa', 'url': 'https://suckhoedoisong.vn/ap-xe-nao-do-amip-nguyen-nhan-trieu-chung-cach-dieu-tri-va-phong-ngua-169240926100943926.htm', 'su

In [128]:
import requests
from bs4 import BeautifulSoup, Tag
import re
import json


def clean_numbering(text):
    """Xoá số thứ tự đầu dòng kiểu 1., 1.1., 2.3.4. trong title."""
    return re.sub(r"^\d+(\.\d+)*[.:]?\s*", "", text).strip()


def normalize_text(text):
    """Chuẩn hoá khoảng trắng và dấu gạch ngang."""
    text = re.sub(r"(?<!\w)-(?!\w)", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


def extract_llm_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    container = soup.find("div", class_="detail-content afcbc-body")
    if not container:
        raise ValueError("Không tìm thấy thẻ nội dung chính.")

    results = []
    current = None

    for tag in container.descendants:
        if not isinstance(tag, Tag):
            continue

        if tag.name == "h2":
            if current:
                results.append(current)
            current = {
                "title": clean_numbering(tag.get_text(strip=True)),
                "content": ""
            }

        elif tag.name in ["h3", "p"] and current:
            full_text = tag.get_text(strip=True)
            if not full_text:
                continue
            current["content"] += ("<sep>" if current["content"] else " ") + full_text

    if current:
        results.append(current)

    return results


if __name__ == "__main__":
    url = "https://suckhoedoisong.vn/dieu-tri-bong-de-the-nao-169240719134922785.htm"
    data = extract_llm_data(url)

    with open("llm_output_cleaned.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print("✅ Done. Output đã được lưu vào llm_output_cleaned.json")


✅ Done. Output đã được lưu vào llm_output_cleaned.json


In [122]:
text = ".1.1. Thuốc chống trầm cảm 3 vòng"
cleaned = re.sub(r"^\d+(\.\d+)*[.:]?\s*", "", text)
cleaned

'.1.1. Thuốc chống trầm cảm 3 vòng'

In [72]:
len(diseases)

16